# List of all emoticons as GIFs

(last updated 2021-06-14)

The code below generates a list of all emoticons in dota and their unicode character. You can copy & paste the character into Dota 2 to use the emoticon. It doesn't matter if you don't own the emoticon pack. This works in Reborn!

In [2]:
from __future__ import unicode_literals
import os
import vdf
from PIL import Image
from collections import OrderedDict
from IPython.display import HTML

out_temp = "./.etmp"
out_dir = "resources/emoticons"
out_sizes = [32, 96]

# steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons.txt' -nd
# steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons/*.vtex_c' -nd -o ./.etmp/cache
data = vdf.loads(open('./emoticons.txt', 'rb').read().decode('utf-16le'), mapper=OrderedDict)

!/d/tools/vrf/Decompiler.exe -i ./$out_temp/cache -o ./$out_temp/cache >/dev/null  # I'm on Cygwin, ok

resp = "<table><tr>" + "<th colspan={}>preview</th><th>chr</th><th>alias</th>".format(len(out_sizes))*3 + "</tr>\n"

for k, v in data['emoticons'].items():   
    k = int(k)
    name = v['image_name'].replace('.', '_') + '.png'
    try:
        f = open("%s/cache/%s" % (out_temp, name), 'rb')
    except IOError:
        continue
    filename = "{0:0>3d}".format(k)
    out_paths = []
    for size in out_sizes:
        out_paths.append(os.path.join(out_dir, "{0}_{1:d}.gif".format(filename, size)))

    # if a gif doesn't exist generate it
    if not all(map(os.path.isfile, out_paths)):   
        !rm -f $out_temp/*.png

        # split the png sequence in seperate files                              
        im = Image.open(f)
        for out_path, size in zip(out_paths, out_sizes):
            for i in range(im.size[0] // 32):
                im.crop((i*32, 0, (i+1)*32, 32))\
                .resize((size, size), Image.NEAREST)\
                .save("{0}/{1:0>3d}.png".format(out_temp, i))
            # combines the sequence images into a gif using imagemagik
            !convert -loop 0 -delay 10 -alpha set -dispose previous "$out_temp/*.png" "$out_path"
    
    # generate table cells for emoticon
    if (int(k) - 1) % 3 == 0:
        resp += "<tr>\n"    
    for path in out_paths:
        resp += "<td><img src='{}'></td>\n".format(path)
    resp += "<td>:{0:s}:</td>\n".format(v['aliases']['0'])
    if (int(k) - 1) % 3 == 2:
        resp += "</tr>\n"

if (int(k) - 1) % 3 != 2:
    resp += "</tr>"
resp += "</table>"

# Emoticon table

In [3]:
HTML(resp)

preview,chr,alias,preview,chr,alias,preview,chr,alias
,,:wink:,,,:blush:,,,:cheeky:
,,:cool:,,,:crazy:,,,:cry:
,,:disapprove:,,,:doubledamage:,,,:facepalm:
,,:happytears:,,,:haste:,,,:hex:
,,:highfive:,,,:huh:,,,:hush:
,,:illusion:,,,:invisibility:,,,:laugh:
,,:rage:,,,:regeneration:,,,:sad:
,,:sick:,,,:sleeping:,,,:smile:
,,:surprise:,,,:aaaah:,,,:burn:
,,:hide:,,,:iceburn:,,,:tears:
